<img src="https://raw.githubusercontent.com/comet-ml/opik/main/apps/opik-documentation/documentation/static/img/opik-logo.svg" width="250"/>

# Tracking a Multi-step LLM Chain

In this exercise, you'll track a multi-step LLM chain with Opik. You can use OpenAI or open source models via LiteLLM.

If you have multiple steps in your LLM pipeline, you can use the `track` decorator to log the traces for each step. If OpenAI is called within one of these steps, the LLM call with be associated with that corresponding step:

# Imports & Configuration

In [1]:
%pip install opik google-genai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.3/149.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.0/418.0 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 4.7 MB/s eta 0:00:00


In [2]:
from opik import track
import opik
from opik.integrations.genai import track_genai
from google import genai
import getpass
import os

os.environ["OPIK_PROJECT_NAME"] = "Multi-step-Chain-Demo"

In [3]:
# opik configs
if "OPIK_API_KEY" not in os.environ:
    os.environ["OPIK_API_KEY"] = getpass.getpass("Enter your Opik API key: ")

opik.configure()

Enter your Opik API key: ··········
Do you want to use "adityadeshpande03" workspace? (Y/n)Y


OPIK: Configuration saved to file: /root/.opik.config


In [4]:
# openai configs
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API key: ")
client = genai.Client()
gemini_client = track_genai(client)

Enter your Google API key: ··········


# Define First Step

In [9]:
@track
def generate_meal(ingredient):
    prompt = f"Generate one example of a meal that can be made with {ingredient}."
    res = client.models.generate_content(model="gemini-2.0-flash", contents=prompt)
    return res.text

# Define Second Step

In [10]:
@track
def generate_recipe(meal):
    prompt = f"Generate a step-by-step recipe for {meal}"
    res = client.models.generate_content(model="gemini-2.0-flash", contents=prompt)
    return res.text

# Call Chain

In [11]:
@track
def generate_recipe_from_ingredient(ingredient):
    meal = generate_meal(ingredient)
    story = generate_recipe(meal)
    return story

generate_recipe_from_ingredient("garlic")

"Okay, that's a great recipe! Here are a few additional suggestions that could enhance it even further:\n\n**Optional Enhancements & Variations:**\n\n*   **More Garlic Techniques:**  Instead of *just* mincing the garlic, consider these options for added depth:\n    *   **Garlic Confit Oil:**  Cook garlic cloves slowly in olive oil over low heat until they're soft and spreadable. Use the oil from the confit for sautéing and stir in a tablespoon of the confit garlic itself towards the end for a richer, sweeter garlic flavor.  You can even make the confit in advance.\n    *   **Roasted Garlic:** Roast a bulb of garlic until soft. Squeeze out the roasted cloves and mash them into a paste. Add a tablespoon of this to the sauce for a mellow, nutty garlic flavor.\n\n*   **Herbs Beyond Parsley:**  While parsley is classic, consider adding or substituting:\n    *   **Chives:**  Their mild oniony flavor complements garlic well.\n    *   **Basil:**  Provides a fresh, slightly sweet note.  Add it 

# Try with your own example!

In [12]:
generate_recipe_from_ingredient(input("Enter an ingredient: "))

Enter an ingredient: Onion


"That's a fantastic and classic recipe for French Onion Soup! The instructions are clear, concise, and easy to follow. The inclusion of tips like being patient with caramelizing the onions and deglazing with wine elevates it. Excellent work!\n\nIf you'd like, I can offer some suggestions for variations or additions:\n\n*   **A touch of Sherry Vinegar:** A splash of sherry vinegar towards the end of the caramelization process can add a lovely tanginess.\n*   **Garlic:** A clove or two of minced garlic added during the last few minutes of caramelizing the onions can add another layer of flavor.\n*   **Brandy or Cognac:** Instead of or in addition to red wine, a tablespoon or two of brandy or cognac can add a richer, more complex flavor.\n*   **Worcestershire Sauce:** A teaspoon of Worcestershire sauce can add umami and depth.\n*   **Different Cheese:** While Gruyere is classic, other cheeses like Emmental, Provolone, or even a mix can be used.\n*   **Garlic Bread Croutons:** Instead of p